### Built-in Tool

#### DuckDuckGo Search

In [1]:
from langchain_community.tools import DuckDuckGoSearchRun

search_tool = DuckDuckGoSearchRun()

results = search_tool.invoke('top news in india today')

print(results)

/Users/daniyalkhan/anaconda3/envs/genAI/lib/python3.10/site-packages/langchain_community/utilities/duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


Check today's top 10 news stories, headline news, breaking news, latest news, politics news, sports news, entertainment news and … 1 day ago · Today's top India news headlines, news on Indian politics, elections, government, business, technology, and Bollywood. Top News Headlines Today: Find current Top Stories in India from India Today. Top news stories from world, politics, business, … 1 day ago · Read all India news and breaking news today from India on politics, business, entertainment, technology, sports, lifestyle … Welcome to The Hindu—your go-to source for today’s breaking news, top headlines, and live updates. Stay informed with the latest from …


#### Built-in Tool - Shell Tool

In [3]:
from langchain_community.tools import ShellTool

shell_tool = ShellTool()

results = shell_tool.invoke('whoami')

print(results)

Executing command:
 whoami
daniyalkhan



/Users/daniyalkhan/anaconda3/envs/genAI/lib/python3.10/site-packages/langchain_community/tools/shell/tool.py:33: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(


### Custom Tools

#### Method 1- Using tool decorator

In [4]:
from langchain_core.tools import tool

In [5]:
# Step 1 - create a function

def multiply(a, b):
    """Multiply two numbers"""
    return a*b

In [6]:
# Step 2 - add type hints

def multiply(a: int, b:int) -> int:
    """Multiply two numbers"""
    return a*b

In [7]:
# Step 3 - add tool decorator

@tool
def multiply(a: int, b:int) -> int:
    """Multiply two numbers"""
    return a*b

In [8]:
result = multiply.invoke({"a":3, "b":5})

In [9]:
print(result)

15


In [10]:
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Multiply two numbers
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


In [11]:
#llm see this instead of function
print(multiply.args_schema.model_json_schema())

{'description': 'Multiply two numbers', 'properties': {'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}, 'required': ['a', 'b'], 'title': 'multiply', 'type': 'object'}


#### Method 2 - Using StructuredTool

In [12]:
from langchain.tools import StructuredTool
from pydantic import BaseModel, Field

In [13]:
class MultiplyInput(BaseModel):
    a: int = Field(required=True, description="The first number to add")
    b: int = Field(required=True, description="The second number to add")

/Users/daniyalkhan/anaconda3/envs/genAI/lib/python3.10/site-packages/pydantic/fields.py:814: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'required'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.8/migration/
  warn(
/Users/daniyalkhan/anaconda3/envs/genAI/lib/python3.10/site-packages/pydantic/fields.py:814: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'required'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.8/migration/
  warn(


In [14]:
def multiply_func(a: int, b: int) -> int:
    return a * b

In [15]:
multiply_tool = StructuredTool.from_function(
    func=multiply_func,
    name="multiply",
    description="Multiply two numbers",
    args_schema=MultiplyInput
)

In [16]:
result = multiply_tool.invoke({'a':3, 'b':3})

print(result)
print(multiply_tool.name)
print(multiply_tool.description)
print(multiply_tool.args)

9
multiply
Multiply two numbers
{'a': {'description': 'The first number to add', 'required': True, 'title': 'A', 'type': 'integer'}, 'b': {'description': 'The second number to add', 'required': True, 'title': 'B', 'type': 'integer'}}


#### Method 3 - Using BaseTool Class

In [17]:
from langchain.tools import BaseTool
from typing import Type

In [18]:
# arg schema using pydantic

class MultiplyInput(BaseModel):
    a: int = Field(required=True, description="The first number to add")
    b: int = Field(required=True, description="The second number to add")

/Users/daniyalkhan/anaconda3/envs/genAI/lib/python3.10/site-packages/pydantic/fields.py:814: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'required'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.8/migration/
  warn(
/Users/daniyalkhan/anaconda3/envs/genAI/lib/python3.10/site-packages/pydantic/fields.py:814: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'required'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.8/migration/
  warn(


In [19]:
class MultiplyTool(BaseTool):
    name: str = "multiply"
    description: str = "Multiply two numbers"

    args_schema: Type[BaseModel] = MultiplyInput

    def _run(self, a: int, b: int) -> int:
        return a * b

In [20]:
multiply_tool = MultiplyTool()

In [21]:
result = multiply_tool.invoke({'a':3, 'b':3})

print(result)
print(multiply_tool.name)
print(multiply_tool.description)

print(multiply_tool.args)

9
multiply
Multiply two numbers
{'a': {'description': 'The first number to add', 'required': True, 'title': 'A', 'type': 'integer'}, 'b': {'description': 'The second number to add', 'required': True, 'title': 'B', 'type': 'integer'}}


### Toolkit

In [22]:
from langchain_core.tools import tool

# Custom tools
@tool
def add(a: int, b: int) -> int:
    """Add two numbers"""
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers"""
    return a * b


In [23]:
class MathToolkit:
    def get_tools(self):
        return [add, multiply]


In [24]:
toolkit = MathToolkit()
tools = toolkit.get_tools()

for tool in tools:
    print(tool.name, "=>", tool.description)


add => Add two numbers
multiply => Multiply two numbers


### Tool Calling

In [25]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests

In [26]:
# tool create

@tool
def multiply(a: int, b: int) -> int:
  """Given 2 numbers a and b this tool returns their product"""
  return a * b

In [28]:
print(multiply.invoke({'a':3, 'b':4}))

12


In [29]:
# tool binding

In [30]:
llm = ChatOpenAI()

In [31]:
llm.invoke('hi')

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 8, 'total_tokens': 17, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BvmkOLFL8KaMowr1qf8zaspsmlFTo', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--99b3313b-5424-4bf5-ad31-fcd21a193520-0', usage_metadata={'input_tokens': 8, 'output_tokens': 9, 'total_tokens': 17, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [32]:
llm_with_tools = llm.bind_tools([multiply])

In [33]:
llm_with_tools.invoke('Hi how are you')

AIMessage(content="I'm just a computer program, so I don't have feelings, but I'm here to help you with any questions or tasks you have. How can I assist you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 58, 'total_tokens': 95, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BvmkYhpNMgC0RmrTJNcPcEmSU51My', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--b3221af5-9eed-4575-bd8c-a86f2d7480cc-0', usage_metadata={'input_tokens': 58, 'output_tokens': 37, 'total_tokens': 95, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [52]:
query = HumanMessage('can you multiply 3 with 1000')
messages = [query]
messages

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={})]

In [53]:
result = llm_with_tools.invoke(messages)
result

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_cR3dpN3PDkCqDSeMWYXyEZRd', 'function': {'arguments': '{"a":3,"b":1000}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 63, 'total_tokens': 81, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-Bvmos0gUQMV8UhIjgxNbOxKh1cBWz', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--249627b0-ee42-4743-a536-55f98bc5b3e9-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 1000}, 'id': 'call_cR3dpN3PDkCqDSeMWYXyEZRd', 'type': 'tool_call'}], usage_metadata={'input_tokens': 63, 'output_tokens': 18, 'total_tokens': 81, 'input_token_details': {'audio': 0, 'cache_read': 

In [54]:
messages.append(result)

In [55]:
messages

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_cR3dpN3PDkCqDSeMWYXyEZRd', 'function': {'arguments': '{"a":3,"b":1000}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 63, 'total_tokens': 81, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-Bvmos0gUQMV8UhIjgxNbOxKh1cBWz', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--249627b0-ee42-4743-a536-55f98bc5b3e9-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 1000}, 'id': 'call_cR3dpN3PDkCqDSeMWYXyEZRd', 'type': 'tool_call'}], usage_metadata={'input_toke

In [56]:
tool_result = multiply.invoke(result.tool_calls[0])

In [57]:
tool_result

ToolMessage(content='3000', name='multiply', tool_call_id='call_cR3dpN3PDkCqDSeMWYXyEZRd')

In [58]:
messages.append(tool_result)
messages

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_cR3dpN3PDkCqDSeMWYXyEZRd', 'function': {'arguments': '{"a":3,"b":1000}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 63, 'total_tokens': 81, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-Bvmos0gUQMV8UhIjgxNbOxKh1cBWz', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--249627b0-ee42-4743-a536-55f98bc5b3e9-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 1000}, 'id': 'call_cR3dpN3PDkCqDSeMWYXyEZRd', 'type': 'tool_call'}], usage_metadata={'input_toke

In [59]:
llm_with_tools.invoke(messages).content

'Yes, the product of 3 and 1000 is 3000.'